## 日英翻訳によるデータ拡張

- JParaCrawl big model v3.0 を使用
- パスは適当に設定してください

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# ライブラリのインストール
! pip install sentencepiece
! pip install fairseq==0.12.1
! pip install sacrebleu
! pip install sacrebleu[ja]

# テストセット(test,dev)であるwmt20のロード
! sacrebleu -t wmt20 -l en-ja --echo src > wmt.test.en
! sacrebleu -t wmt20 -l en-ja --echo ref > wmt.test.ja
! sacrebleu -t wmt20/dev -l en-ja --echo src > wmt.valid.en
! sacrebleu -t wmt20/dev -l en-ja --echo ref > wmt.valid.ja

In [ ]:
!wget 'https://www.kecl.ntt.co.jp/icl/lirg/jparacrawl/release/1.0/pretrained_models/en-ja/big.tar.gz'
!tar -zxvf big.tar.gz
!wget 'https://www.kecl.ntt.co.jp/icl/lirg/jparacrawl/release/3.0/spm_models/en-ja_spm.tar.gz'
!tar -zxvf en-ja_spm.tar.gz
!wget 'https://data.statmt.org/news-crawl/ja/news.2021.ja.shuffled.deduped.gz'
!tar -zxvf news.2021.ja.shuffled.deduped.gz

In [ ]:
import re
import unicodedata
import sentencepiece as spm

# Fairseq

In [ ]:
# 準備
# unkタグの割合が示してある
# --destdir :バイナリデータの保存場所(def:data-bin)
ENDICT='big-3.0-jaen/dict.en.txt'
JADICT='big-3.0-jaen/dict.ja.txt'
! fairseq-preprocess --source-lang ja --target-lang en --trainpref train --validpref valid --testpref test \
                     --destdir pbl-aug --srcdict $JADICT --tgtdict $ENDICT

In [ ]:
# 訓練
# データ拡張なので、無視してもいい
PRETRAINED_MODEL="/content/drive/MyDrive/big-3.0-jaen/big.pretrain.pt"
SEED=10
! fairseq-train pbl-aug --arch transformer --restore-file $PRETRAINED_MODEL \
    --no-epoch-checkpoints --no-last-checkpoints\
    --seed $SEED \
    --patience 10 \
    --batch-size 16 --optimizer adam --adam-betas '(0.9,0.98)' --lr 1e-4 --lr-scheduler inverse_sqrt --warmup-updates 4000 --warmup-init-lr 1e-07 \
    --dropout 0.1 --weight-decay 0.0001 --clip-norm 1.0 \
    --reset-optimizer --reset-meters --reset-dataloader --reset-lr-scheduler \
    --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
    --encoder-embed-dim 1024 --decoder-embed-dim 1024 \
    --encoder-ffn-embed-dim 4096 --decoder-ffn-embed-dim 4096 \
    --encoder-attention-heads 16 --decoder-attention-heads 16 \
    --log-interval 100 --validate-interval-updates 9999 --save-interval-updates 1000

### データ拡張

In [ ]:
!mv news.2021.ja.shuffled.deduped news-crawl21-500k.ja

In [ ]:
MODEL='big-3.0-jaen/big.pretrain.pt'

# 評価
# max-len-b :出力単語数の制限
IN_FILE='news.2021.ja.shuffled.deduped'
# pbl-big3-enja 23.1
! fairseq-interactive pbl-aug --path $MODEL --input $REF_FILE --batch-size 128 \
 --remove-bpe sentencepiece --buffer-size 1024 --nbest 1 --max-len-b 50\
--beam 5 --task translation\
| grep "^H-" | sort -V | cut -f3 > news-crawl21-500k.en